In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import torch
import matplotlib.pyplot as plt
import numpy as np
from vibromodes.run import load_model,load_data,load_configs,set_global_seed
from vibromodes.metrics import generate_frfs
from vibromodes.plotting import compare_fields_side_by_side
from vibromodes.hdf5_dataset import load_testset
from vibromodes.velocity_field import field_dict2frf
from pathlib import Path
import seaborn
import pandas as pd

In [4]:

from vibromodes.models import ModeONet


set_global_seed(42)


model = ModeONet(32,16)

model.eval()
model.load_state_dict(torch.load(f"./model.pth"))

device = torch.device("cuda")

model = model.to(device)

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [ ]:
test_set_path = "<...>/D500_edit.h5"

dataset = load_testset(test_set_path)

In [ ]:
dataloader = torch.utils.data.DataLoader(dataset,batch_size=8,shuffle=False,drop_last=False,collate_fn=lambda x: x)

In [ ]:
dataset.pattern.shape

In [ ]:
from vibromodes.metrics import generate_frfs,calc_metrics

#FQO: 5 min #131 s ohne comp
# 0.36 #15 ohne comp

frfs = generate_frfs(model,dataloader,device,tqdm_enable=True)
frf_results = calc_metrics(frfs.pred_frfs,frfs.target_frfs)

In [ ]:
frame = pd.DataFrame()
for key in frf_results:
    frame[key] = [frf_results[key]]

frame

In [ ]:
modeonet_color = "#F9A36C"
plt.xlabel("Frequency (dB)")
plt.ylabel("Velocity (dB)")
plt.plot(frfs.target_frfs[0].detach().cpu(),color="black")
plt.plot(frfs.pred_frfs[0].detach().cpu(),color=modeonet_color,linstyle="--")